In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import sys

In [2]:
class PDB_NERDSS():
    
    def __init__(self, parentDir='', pdbWrite=1e5):
        self._pdir = parentDir
        self._pdbStep = pdbWrite
        self.t = []
        self.x = []
        self.r = []
        self.proteinNumbers = []
        self.left = []
        self.right = []
        self.t_longest = []
        self.x_all = []
        self.x_onDNA = []
    
    def read_pdbs(self, N_repeat=None, pdbMin=0, pdbMax=np.inf, YZcenter=31.623/2, timeStep:float=1e-6):
        '''Read pdbs untill pdbMax
        
        Args:
        
            N_repeat (int, iterable): 
                (int) the number of repeats, will read subfolders named in `range(N_repeat)`
                (iterable of strings) the names of subfolders
            
            pdbMin (int): The min index (number of steps) of pdb read.
            
            pdbMax (int): The max index (number of steps) of pdb read.
            
            YZcenter (float): The center coordinate of Y and Z axes (they are the same) used to calculate r.
            
            timeStep (float): time jump for one step.
            
        Returns:
            None: 
                self.x (list of lists): x coordinates
                self.r (list of lists): r distances
                self.t (list of lists): time points
                self.t_longest (list): the longest time points
        '''
        
        self._pdbMax = pdbMax
        self._YZcenter = YZcenter
        self._tStep = timeStep
        
        if N_repeat is None:
            self._subnames = ['']
        elif isinstance(N_repeat, int):
            self._subnames = [str(i) for i in range(N_repeat)]
        else: # assume it is iterable
            self._subnames = [str(i) for i in N_repeat]
        self._Nrepeats = len(self._subnames)
        
        for subname in self._subnames:
            x = []
            r = []
            t = []
            NItr = pdbMin # include the initial coordinates
            while NItr < self._pdbMax:
                # in case the simulation does not run through pdbMax*pdbStep
                filename = int(NItr*self._pdbStep)
                x_itr = []
                r_itr = []
                try:
                    with open(f'{self._pdir}/{subname}/PDB/{filename}.pdb') as f:
                        for line in f:
                            linelist = line.split()
                            if linelist[2] == 'COM' and linelist[3] == 'P':
                                x_itr.append(float(linelist[5]))
                                CRDy = float(linelist[6])
                                CRDz = float(linelist[7])
                                r_itr.append(
                                    ((CRDy-self._YZcenter)**2 + (CRDz-self._YZcenter)**2)**0.5
                                )
                    t.append(NItr*self._pdbStep*self._tStep)
                    NItr += 1
                except:
                    break
                x.append(x_itr)
                r.append(r_itr)
            self.x.append(x)
            self.r.append(r)
            self.t.append(t)
        
        self._subnames = [name for i, name in enumerate(self._subnames) if len(self.t[i]) != 0]
        self._Nrepeats = len(self._subnames)
        self.x = [x for i, x in enumerate(self.x) if len(self.t[i]) != 0]
        self.r = [r for i, r in enumerate(self.r) if len(self.t[i]) != 0]
        self.t = [t for t in self.t if len(t)!=0]
                
        
        self.t_longest = self.t[0]
        for t in self.t:
            if len(t) > len(self.t_longest):
                self.t_longest = t
        self.left = np.zeros_like(self.t_longest)
        self.right = np.zeros_like(self.t_longest)
    
    def classify_proteins(self, burnInSteps=1, windowMax=10, chromRadius=5.5, Xcenter=64/2):
        '''Classify proteins into different regions
        
        Args:
            burnInSteps (int): The min number of pdbs considered. Use this to exclude burn-in steps.
            
            windowMax (int): 
                The max window size used for calculating avarages. 
                There should be windowMax >= burnInSteps, otherwise windowMax = burnInSteps
            
            chromRadius (float): 
                The radius of chromatin. When r < chromRadius, this protein is considered as lying on DNA.
                Set this to np.inf to count all proteins in solution.
            
            Xcenter (float): The center of X axis. 
            
        Returns:
            None: 
                self.left (1darray): the probabilities of proteins in left
                self.right (1darray): the probabilities of proteins in right
                They are normalized to the total number of proteins.
        '''
        
        self._pdbNumMin = burnInSteps
        self._windowMax = max(windowMax, burnInSteps)
        self._chromR = chromRadius
        self._Xcenter = Xcenter
        
        left = []
        right = []
        self._copies = np.zeros_like(self.t_longest)
        for nr in range(self._Nrepeats):
            xn = np.array(self.x[nr])
            rn = np.array(self.r[nr])
            leftn = np.zeros(xn.shape[0])
            rightn = np.zeros(xn.shape[0])
            numPs = xn.shape[1]
            if numPs == 0:
                pass
            else:
                for ip in range(numPs):
                    leftn[ (xn[:,ip] < Xcenter) & (rn[:,ip] < chromRadius) ] += 1
                    rightn[ (xn[:,ip] > Xcenter) & (rn[:,ip] < chromRadius) ] += 1
                leftn[:self._pdbNumMin] = leftn[self._pdbNumMin]
                rightn[:self._pdbNumMin] = rightn[self._pdbNumMin]
                # now, left and right are the probabilities at the relavant timepoint
                left.append(leftn/numPs)
                right.append(rightn/numPs)
        
        for NItr in range(0, len(self.t_longest)):
            leftNItr = [ np.mean(left[nr][max(0,NItr-self._windowMax+1) : NItr+1]) for nr in range(self._Nrepeats) ]
            rightNItr = [ np.mean(right[nr][max(0,NItr-self._windowMax+1) : NItr+1]) for nr in range(self._Nrepeats) ]
#             print(leftNItr)
            self.left[NItr] = np.mean(leftNItr)
            self.right[NItr] = np.mean(rightNItr)

    def find_proteins_on_DNA(self, burnInSteps=0, chromRadius=5.5):
        '''Find proteins that are close to DNA
        
        Args:
            burnInSteps (int): The min number of pdbs considered. Use this to exclude burn-in steps.
            
            chromRadius (float): 
                The radius of chromatin. When r < chromRadius, this protein is considered as lying on DNA.
                Set this to np.inf to count all proteins in solution.
            
        Returns:
            None: 
                self.x_onDNA (1darray): the x coordinates of proteins on DNA
                self.totNumProtein (int): the total number of proteins
        '''
        
        self._pdbNumMin = burnInSteps
        self._chromR = chromRadius
        
        self.x_onDNA = []
        self.totNumProtein = 0
        
        for nr in range(0, self._Nrepeats):
            r_nr = np.array(self.r[nr])
            x_nr = np.array(self.x[nr])
            numPs = x_nr.shape[1]
            for ip in range(numPs):
                x_nr_ip = x_nr[self._pdbNumMin:, ip]
                r_nr_ip = r_nr[self._pdbNumMin:, ip]
                self.x_onDNA += [x for x in x_nr_ip[r_nr_ip < self._chromR]]
                self.totNumProtein += len(x_nr_ip)
            
    def get_all_x(self, burnInSteps=1):
        '''collect all x coordinates of proteins
        
        Args:
            burnInSteps (int): The min number of pdbs considered. Use this to exclude burn-in steps.
            
        Returns:
            self.x_all (list): The x coordinates of proteins
        '''
        
        self._pdbNumMin = burnInSteps
        self.x_all = []
        
        for nr in range(0, self._Nrepeats):
            self.x_all += [x for x in self.x[nr][self._pdbNumMin:]]
        
        return self.x_all
        

In [3]:
def plothist(data, binEdges, normalization, color, ax, style='stairs'):
    hist, bin_edges = np.histogram(data, bins=binEdges, density=False)
    if style=='stairs':
        ax.stairs(hist/normalization, bin_edges, fill=True, color=color)
    elif style=='curve':
        ax.plot((bin_edges[1:]+bin_edges[:-1])/2, hist/normalization, color=color)
        
def plotSpatialDistributions(
    data, burnIn, ax,
    binEdgesOnDNA=np.arange(0+5.5,64-5.5,1),
    binEdgesAll=np.arange(0.5,64,1),
    axLineW=1.5, 
):
    for axis in ['top', 'bottom', 'left', 'right']:
        ax.spines[axis].set_linewidth(axLineW)
    plothist(
        data.x_onDNA, binEdgesOnDNA, 
        data.totNumProtein, 
        'tab:blue', ax, style='stairs'
    )
    plothist(
        data.get_all_x(burnIn), binEdgesAll, 
        data.totNumProtein, 
        'tab:red', ax, style='curve'
    )

    plt.semilogy()

    plt.xlim([0.5,64-0.5])
    plt.xticks([15, 17, 43, 52], ['S ', ' S', 'S', 'S'], fontsize=18)
    plt.ylim([1e-3,1e0])
    plt.yticks(fontsize=18)

In [4]:
def drawHists(
    pdbs_1, pdbs_2, ylim, burnIn, xlength, histWidth=1, figsize=(16,2),
    SCrds = [-39.0, -37.0, -35.0, 18.5, 37.0, 55.5],
    SLabels = ['', 'SSS', '', 'S', 'S', 'S'], 
    offset=0, fontsize=18, linewidth=1.5, tickL=10, tickW=2,
):
    plt.figure(figsize=figsize)
    plt.subplots_adjust(wspace=0)
    ax = plt.subplot(121)

    for axis in ['top', 'bottom', 'left', 'right']:
        ax.spines[axis].set_linewidth(linewidth)
    plt.tick_params(length=tickL, width=tickW)
    
    plothist(
        pdbs_1.x_onDNA, np.arange(0+5.5+offset,xlength-5.5,histWidth), 
        pdbs_1.totNumProtein, 
        'tab:blue', ax, style='stairs'
    )
    plothist(
        pdbs_1.get_all_x(burnIn), np.arange(0.5+offset,xlength,histWidth), 
        pdbs_1.totNumProtein, 
        'tab:red', ax, style='curve'
    )

    plt.semilogy()
    plt.xlim([0.5,xlength-0.5])
    plt.xticks(np.array(SCrds)+xlength/2, SLabels, fontsize=fontsize)
    plt.ylim(ylim)
    plt.yticks(fontsize=fontsize)

    ax = plt.subplot(122)
    for axis in ['top', 'bottom', 'left', 'right']:
        ax.spines[axis].set_linewidth(linewidth)
    plt.tick_params(length=tickL, width=tickW)
    plothist(
        pdbs_2.x_onDNA, np.arange(0+5.5+offset,xlength-5.5,histWidth), 
        pdbs_2.totNumProtein, 
        'tab:blue', ax, style='stairs'
    )
    plothist(
        pdbs_2.get_all_x(burnIn), np.arange(0.5+offset,xlength,histWidth), 
        pdbs_2.totNumProtein, 
        'tab:red', ax, style='curve'
    )
    
    plt.semilogy()
    plt.xlim([0.5,xlength-0.5])
    plt.xticks(np.array(SCrds)+xlength/2, SLabels, fontsize=fontsize)
    plt.ylim(ylim)
    plt.yticks([])

In [5]:
slcList_repeats = []
repeatID_pool = np.arange(0, 48, 1)

def selectivity(pdbs, xlength):
    pdbs.x_onDNA = np.array(pdbs.x_onDNA)
    numProteins_right = np.sum(pdbs.x_onDNA > xlength/2)
    if numProteins_right == 0:
        return len(pdbs.x_onDNA) / 1
    else:
        return (np.sum(pdbs.x_onDNA < xlength/2)/numProteins_right)

def bootStrapSelectivity(pdbs, xlength, N_batch=100):
    pdbs.x_onDNA = np.array(pdbs.x_onDNA)
    N_sample = pdbs.x_onDNA.shape[0]
    # print(N_sample, batchSize)
    selectivityList = []
    for _ in range(N_batch):
        randomChoices = np.random.choice(pdbs.x_onDNA, size=N_sample, replace=True)
        numProteins_right = np.sum(randomChoices > xlength/2)
        if numProteins_right == 0:
            selectivityList.append(N_sample / 1) 
        else:
            selectivityList.append(np.sum(randomChoices < xlength/2)/numProteins_right)
    return selectivityList

In [6]:
simDirs = 'kpp0kpnE1.0 kppE1.0kpnE1.0  kppE2.0kpnE1.0  kppE3.0kpnE1.0  kppE4.0kpnE1.0  kppE5.0kpnE1.0  kppE6.0kpnE1.0'.split()
PDBS = []
burnIn = 0
xlength = 106
slcList = []
for simdir in simDirs:
    print(simdir, end=' ')
    PDBS.append(PDB_NERDSS(f'./{simdir}/', 6e7))
    PDBS[-1].read_pdbs(N_repeat=repeatID_pool, pdbMin=2)
    PDBS[-1].find_proteins_on_DNA(burnIn)
    slcList.append(selectivity(PDBS[-1], xlength))

kpp0kpnE1.0 kppE1.0kpnE1.0 kppE2.0kpnE1.0 kppE3.0kpnE1.0 kppE4.0kpnE1.0 kppE5.0kpnE1.0 kppE6.0kpnE1.0 

In [7]:
print(slcList)

[0.7746478873239436, 1.5805084745762712, 2.3769470404984423, 3.0035714285714286, 2.7454545454545456, 2.28625, 3.410923276983095]


In [11]:
selectivity_list = []
for pdb in PDBS:
    selectivity_list.append(bootStrapSelectivity(pdb, xlength, 100))
print(*np.mean(selectivity_list, axis=1), sep=', ')
print(*np.std(selectivity_list, axis=1), sep=', ')

0.7839380458445123, 1.608361702731444, 2.4142460159166124, 3.017057706286011, 2.75540309837026, 2.2933347821954073, 3.4153309304606063
0.06433321086749408, 0.12193825141052676, 0.15954401742176008, 0.21645287528201054, 0.12229826213536317, 0.09116039107861558, 0.14586540127521788
